In [ ]:
import sys
import os 
import tensorflow as tf 
from PIL import Image  #注意Image,后面会用到
import matplotlib.pyplot as plt 
import numpy as np


# sys.path.insert(0, '../models/slim/')  models-master research
#sys.path.insert(0, './models/research/slim/') #把后面的路径插入到系统路径中 idx=0
from datasets import dataset_utils
import math
import os
import tensorflow as tf

#教學
## https://blog.csdn.net/miaomiaoyuan/article/details/56865361
## https://tw.saowen.com/a/42f75dc906d3ae05c05d1492b6b5f371724891ab774b21db02d348175cb2fe60

#classes={'smo','nosmo'} # 设定 2 类 smo ,nosmo
classes={'nosmo','smo'} # 设定 2 类 smo ,nosmo

#根据list路径 ,把数据转化为TFRecord


def convert_dataset_from_train( data_dir, output_dir):  
    
    for index, name in enumerate(classes):
        
        class_path = data_dir + name # #每一類圖片的地址:smo or nosmo
        
        #class_path ='/home/aewin/work/anaconda3/code/V001/smorking/train'+'/'+name
        
        current = 0
        
        for img_name in os.listdir(class_path): 
            

            img_path = class_path + '/' + img_name ##每一張圖片的路徑  single image
            
            print(img_path)
            print(current)
            
            current +=1
            
            img1 = Image.open(img_path)
            
            img1 = img1.resize((299,299))
            
            img_raw = img1.tobytes()#将图片转化为二进制格式
            
            #example 对象 对 label 和 image 数据进行封装
            example = tf.train.Example(features=tf.train.Features( feature = {
                "label": tf.train.Feature(int64_list = tf.train.Int64List(value = [index])),
                'img_raw': tf.train.Feature(bytes_list = tf.train.BytesList(value = [img_raw]))
            }))
            
            writer.write(example.SerializeToString())  #序列化为字符串  #串行化為字符串
            
    writer.close()

    
writer= tf.python_io.TFRecordWriter("./smorking_train_1.tfrecords") #要生成的文件   
convert_dataset_from_train('/home/aewin/work/anaconda3/code/V001/smorking/train/', './train_out_tf/')

#os.system('mkdir -p val')


In [ ]:
## 然後讀取生成的tfrecords數據，並且將tfrecords裏面的數據保存成jpg格式的圖片。具體代碼如下：


def read_and_decode(filename): # 读入dog_train.tfrecords

    #根據文檔名生成一個隊列
    filename_queue = tf.train.string_input_producer([filename])#生成一个queue队列

    reader = tf.TFRecordReader()
    
    _, serialized_example = reader.read(filename_queue)#返回文件名和文件 将image数据和label取出来
    
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })#将image数据和label取出来

    rimg = tf.decode_raw(features['img_raw'], tf.uint8)
    rimg = tf.reshape(rimg, [299, 299, 3])  #reshape为299 * 299 的3通道图片 
    rimg = tf.cast(rimg, tf.float32) * (1. / 255) - 0.5  #將圖片變成tensor    ,#在流中抛出img张量
                                                       #對圖片進行歸一化操作將【0，255】之間的像素歸一化到【-0.5，0.5】，標準化處理可以使得不同的特徵具有相同的尺度（Scale）。
                                                       #這樣，在使用梯度下降法學習參數的時候，不同特徵對參數的影響程度就一樣了
            
    label = tf.cast(features['label'], tf.int32)#將標籤轉化tensor
    print (rimg)
    print (label)
    return rimg, label




img, label = read_and_decode('/home/aewin/work/anaconda3/code/V001/smorking_train.tfrecords')

#形成一個batch的數據，由於使用shuffle，因此每次取batch的時候
#都是隨機取的，可以使樣本儘可能被充分地訓練，保證min_after值小於capacit值
#rimg, label = read_and_decode('/home/aewin/work/anaconda3/code/V001/smorking_train.tfrecords')

# img_batch, label_batch = tf.train.shuffle_batch([rimg,label],batch_size=10,capacity=2000,min_after_dequeue=1000)

In [ ]:

## 显示tfrecord格式的图片 把tfrecoder 格式的檔案讀成 圖片檔jpeg 並存在硬碟

init = tf.global_variables_initializer()

def save_tfrecord_jpeg(tfrecord_location,output_banner):

    
    filename_queue = tf.train.string_input_producer([tfrecord_location]) #读入流中

    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件

    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })  #取出包含image和label的feature对象

    image = tf.decode_raw(features['img_raw'], tf.uint8)
    image = tf.reshape(image, [299, 299, 3])
    label = tf.cast(features['label'], tf.int32)
    
    with tf.Session() as sess: #开始一个会话
        
        init_op = tf.initialize_all_variables()# 這步是為了將上面已初使化好的變數準備丟到 sess.run()中
        
        sess.run(init_op)# 創建一個協調器，管理線程 # 這步是為了 叫sess.run去跑詳細觀念；https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/2-3-session/
        
        coord=tf.train.Coordinator() # 啟動QueueRunner, 此時文檔名隊列已經進隊
        
        threads= tf.train.start_queue_runners(coord=coord)
     
        for i in range(5000):  #！寫5000張就好！fot test
            
            example, l = sess.run([image,label])#在会话中取出image和label

            img=Image.fromarray(example, 'RGB')#这里Image是之前提到的

            img.save( './out/'+output_banner+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
            print(example, l,'./out/'+output_banner+str(i)+'_''Label_'+str(l)+'.jpg')
            
        coord.request_stop()# 通知其他線程關閉
        coord.join(threads) # 其他所有線程關閉之後，這一函數才能返回
        
        
save_tfrecord_jpeg('/home/aewin/work/anaconda3/code/V001/smorking_train_1.tfrecords','out_')

###  強制轉換~原本nosmo 無法到299 299用以下CODE將原本image 轉成299 在丟給 tf.train.Example


In [ ]:
###################強制轉換~原本nosmo 無法到299 299用以下CODE將原本image 轉成299 在丟給 tf.train.Example

import os 
import tensorflow as tf 
from PIL import Image  #注意Image,后面会用到
import matplotlib.pyplot as plt 
import numpy as np

cwd='./out/' 
size = 299, 299

classes={'nosmo','smo'} # 设定 2 类 smo ,nosmo

writer= tf.python_io.TFRecordWriter("./nosmo.tfrecords") #要生成的文件

###################強制轉換~原本nosmo 無法到299 299用以下CODE 強轉

class_path='/home/aewin/work/anaconda3/code/V001/out/'+'nosmo'+'/'
current = 0
for img_name in os.listdir(class_path): 
    img_path=class_path+img_name #每一个图片的地址
    current += 1
    img=Image.open(img_path)
    img=Image.open(img_path).convert('RGB')
    img= img.resize((299,299), Image.ANTIALIAS)
    ssstr='/home/aewin/work/anaconda3/code/V001/out/nosmo/'+str(current)+'.jpg' #另存新圖
    #print(ssstr)
    img.save(ssstr)

    print(current)
    img_raw=img.tobytes()#将图片转化为二进制格式
    example = tf.train.Example(features=tf.train.Features(feature={
        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
        'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
    })) #example对象对label和image数据进行封装
    writer.write(example.SerializeToString())  #序列化为字符串

writer.close()
